In [1]:
# Zach Ellenberg, Willard Chan, Rachel "Bum Rat" Kirschner, Spencer Jaquish, Franz Arend
# zellenberg, willchan22, kirschdog, sjaquish, fja1995
# June 3rd, 2016
# Creating Databases with PyGreSQL

In [2]:
import sys
sys.path.append(sys.path.append('C:\Python27\ArcGIS10.3\Lib\site-packages'))
import pg
from pg import DB
import csv

In [3]:
# Connect to PostgreSQL Database
db = DB(dbname = 'geog465_zellenbe', host = 'geog-db2.geog.uw.edu', port = 5432, user = 'zellenbe', passwd = 'zellenbe')

In [4]:
# Drop all of the tables to be able to test code many times
tableList = ['cultural_spaces', 'discipline', 'neighborhood', 'constituency']
for table in tableList:
    db.query("DROP TABLE " + table)

In [5]:
# Create all four tables
db.query("""CREATE TABLE cultural_spaces (
    id int  NOT NULL,
    name varchar  NOT NULL,
    phone varchar  NULL,
    sq_ft_total int  NULL,
    address varchar  NULL,
    location varchar  NOT NULL,
    dom_disc_id int  NOT NULL,
    sec_disc_id int  NOT NULL,
    third_disc_id int  NOT NULL,
    neighborhood_id int  NOT NULL,
    constituency_id int  NOT NULL,
    lat float NULL,
    lng float NULL,
    CONSTRAINT cultural_spaces_pk PRIMARY KEY (id))""")

db.query("""CREATE TABLE discipline (
    id int  NOT NULL,
    discipline varchar  NOT NULL,
    CONSTRAINT discipline_pk PRIMARY KEY (id))""")

db.query("""CREATE TABLE neighborhood (
    id int  NOT NULL,
    neighborhood varchar  NOT NULL,
    CONSTRAINT neighborhood_pk PRIMARY KEY (id))""")

db.query("""CREATE TABLE constituency (
    id int  NOT NULL,
    constituency varchar  NOT NULL,
    CONSTRAINT constituency_pk PRIMARY KEY (id))""")

# Create foreign keys
db.query("""ALTER TABLE cultural_spaces ADD CONSTRAINT Cultural_Spaces_Dominant_Discipline
    FOREIGN KEY (dom_disc_id)
    REFERENCES discipline (id)  
    NOT DEFERRABLE 
    INITIALLY IMMEDIATE""")

db.query("""ALTER TABLE cultural_spaces ADD CONSTRAINT Cultural_Spaces_Second_Discipline
    FOREIGN KEY (sec_disc_id)
    REFERENCES discipline (id)  
    NOT DEFERRABLE 
    INITIALLY IMMEDIATE""")

db.query("""ALTER TABLE cultural_spaces ADD CONSTRAINT Cultural_Spaces_Third_Discipline
    FOREIGN KEY (third_disc_id)
    REFERENCES discipline (id)  
    NOT DEFERRABLE 
    INITIALLY IMMEDIATE""")

db.query("""ALTER TABLE cultural_spaces ADD CONSTRAINT cultural_spaces_constituency
    FOREIGN KEY (constituency_id)
    REFERENCES constituency (id)  
    NOT DEFERRABLE 
    INITIALLY IMMEDIATE""")

db.query("""ALTER TABLE cultural_spaces ADD CONSTRAINT cultural_spaces_neighborhood
    FOREIGN KEY (neighborhood_id)
    REFERENCES neighborhood (id)  
    NOT DEFERRABLE 
    INITIALLY IMMEDIATE""")

In [6]:
# Populate discipline table with data from csv
d_file = open('U:\\465 CSVs\\discipline.csv')
d_reader = csv.reader(d_file, delimiter = ',')

d_reader.next() # Skip header row

for d_row in d_reader:
    str_row = str(d_row)
    str_row = str_row[1:len(str_row)-1]
    db.query("INSERT INTO discipline VALUES (" + str_row + ")")
    
d_file.close()

In [7]:
# Populate neighborhood table with data from csv
n_file = open('U:\\465 CSVs\\neighborhood.csv')
n_reader = csv.reader(n_file, delimiter = ',')

n_reader.next() # Skip header row

for n_row in n_reader:
    str_row = str(n_row)
    str_row = str_row[1:len(str_row)-1]
    db.query("INSERT INTO neighborhood VALUES (" + str_row + ")")
    
n_file.close()

In [8]:
# Populate constituency table with data from csv
c_file = open('U:\\465 CSVs\\constituency.csv')
c_reader = csv.reader(c_file, delimiter = ',')

c_reader.next() # Skip header row

for c_row in c_reader:
    str_row = str(c_row)
    str_row = str_row[1:len(str_row)-1]
    db.query("INSERT INTO constituency VALUES (" + str_row + ")")
    
c_file.close()

In [9]:
# Populate cultural_spaces table with data from csv
cs_file = open('U:\\465 CSVs\\cultural spaces.csv')
cs_reader = csv.reader(cs_file, delimiter = ',')

cs_reader.next() # Skip header row

for cs_row in cs_reader:
    str_row = ''                         # Master string to be used in the query
    for i in range(0,11):                # There are ten columns, so there are 10 values in each row
        val = cs_row[i]
        if i in (1,2,4,5):               # Fields in columns 1, 2, 4 & 5 contain strings
            val = '\'' + val + '\''      # So we put single quotes around them
        elif len(val) == 0:              # Fields that are in other columns are integers, here we check if they are null
            val = '-1'                   # If they are null, they are assigned a value of -1 because there has to be a value
        val = val + ', '                 # We then add a comma to separate the values
        str_row = str_row + (val)        # We add the new value and comma and space to the master string created above
    str_row = str_row[:len(str_row)-2]   # Here we cut off the final comma and space because they are not needed
    db.query("INSERT INTO cultural_spaces VALUES (" + str_row + ")")
    
cs_file.close()

In [10]:
# Here we are filling in the foreign key values and splitting the location field into separate lat and lng fields
# orig means that the variable is referencing the original csv file downloaded from data.seattle.gov
cs_orig_file = open('U:\\Seattle_Cultural_Space_Inventory.csv')
cs_orig_reader = csv.reader(cs_orig_file, delimiter = ',')

cs_orig_reader.next() # Skip header row

for row in cs_orig_reader:
    # Here we are getting the original values for name and phone number, in order to later compare against the database to make
    # sure that we are updating the foreign keys in the correct row
    name_orig = row[0]
    phone_orig = row[1]
    loc_orig = row[40]
    
    # Here we are getting the original values for neighborhood, dominant discipline, second discipline, third discipline
    # and constituency in order to match it to each of their respective tables to obtain the ids to be used as foreign keys
    neighborhood_orig = row[4]
    dom_disc_orig = row[6]
    sec_disc_orig = row[7]
    third_disc_orig = row[8]
    constituency_orig = row[35]
    
    # Here we are getting the id integers for each of the aforementioned values from their respective tables
    neighborhood_for_id_query = "SELECT id FROM neighborhood WHERE neighborhood = \'" + neighborhood_orig + "\'"
    neighborhood_for_id = db.query(neighborhood_for_id_query).getresult()[0][0]
    dom_disc_for_id_query = "SELECT id FROM discipline WHERE discipline = \'" + dom_disc_orig + "\'"
    dom_disc_for_id = db.query(dom_disc_for_id_query).getresult()[0][0]
    sec_disc_for_id_query = "SELECT id FROM discipline WHERE discipline = \'" + sec_disc_orig + "\'"
    sec_disc_for_id = db.query(sec_disc_for_id_query).getresult()[0][0]
    third_disc_for_id_query = "SELECT id FROM discipline WHERE discipline = \'" + third_disc_orig + "\'"
    third_disc_for_id = db.query(third_disc_for_id_query).getresult()[0][0]
    constituency_for_id_query = "SELECT id FROM constituency WHERE constituency = \'" + constituency_orig + "\'"
    constituency_for_id = db.query(constituency_for_id_query).getresult()[0][0]
    
    # Here we compare the name and phone numbers from top of this loop to the ones in the cultural_spaces table,
    # then we update the foreign keys to the id integers generated above
    all_for_id_update_query = ("UPDATE cultural_spaces SET neighborhood_id = " + str(neighborhood_for_id) + ", dom_disc_id =" 
                               + str(dom_disc_for_id) + ", sec_disc_id = " + str(sec_disc_for_id) + """, third_disc_id = 
                               """ + str(third_disc_for_id) + ", constituency_id = " + str(constituency_for_id) + 
                               " WHERE name = \'" + str(name_orig) + "\' AND phone = \'" + str(phone_orig) + "\'")
    db.query(all_for_id_update_query)
    
    # Here we are going to split the lat long field into two fields of type int
    if len(loc_orig) != 0:
        split = loc_orig.split(',')
        lat = split[0][1:]
        lng = split[1][1:-1]
        lat_update_query = ("UPDATE cultural_spaces SET lat = " + lat + " WHERE name = \'" + str(name_orig) + 
                            "\' AND phone = \'" + str(phone_orig) + "\'")
        lng_update_query = ("UPDATE cultural_spaces SET lng = " + lng + " WHERE name = \'" + str(name_orig) + 
                            "\' AND phone = \'" + str(phone_orig) + "\'")
        db.query(lat_update_query)
        db.query(lng_update_query)

cs_orig_file.close()